### Vorticity

Navier Stokes for incompressible flow:

$$\frac{\partial u}{\partial t} + u \circ \nabla u = - \frac{1}{\rho} \nabla p + \nu \nabla ^2 u$$

Non-dimensional:

$$Re \left( \frac{\partial u ^{*}}{\partial t} + u^{*} \circ \nabla u^{*} \right) = - \nabla p^{*} + \nabla ^2 u^{*}$$

$u^{*}$ and $p^{*}$ are the non-dimensional velocity and pressure.

To obtain stokes flow we assume the reynolds number approaches zero:

$$0 = - \nabla p + \nabla ^2 u$$

Curl on both sides:

$$\nabla \times 0 = \nabla \times (-\nabla p + \nabla ^2 u)$$

LHS remains zero, first term on the RHS is:

$$\nabla \times - \nabla p = 0$$

Because $\nabla \times \nabla \phi = 0$ where $\phi$ is a scalar

Finally:

$$\nabla \times \nabla ^2 u = \nabla ^2 \omega$$

Where $\nabla \times u = \omega$ is vorticity

Combining everything we arrive at the simplified vorticity transport eqn for Stokes flow:

$$\nabla ^2 \omega = 0$$

### Stream function

Def stream func $\psi$ such that:

$$u = \frac{\partial \psi}{\partial y} \ and \ v = -\frac{\partial \psi}{\partial x}$$

In 2D we can write out the vorticity as:

$$\omega = \frac{\partial v}{\partial x} - \frac{\partial u}{\partial y}$$

Combined with the previous equation yields:

$$\nabla ^2 \psi = - \omega$$

System of two couple equations that can describe the fluid flow in a liquid driven cavity at very low Reynolds numbers:

$$\nabla ^2 \omega = 0$$
$$\nabla ^2 \psi = - \omega$$

Substituting we get:

$$\nabla ^4 \psi = 0$$

## Cavity flow

Lid of a square cavity moves at constant velocity of $u = 1$ with no fluid leaking out.  All of the surfaces have no-slip BCs.

BCs:

top : $\psi = 0$ and $\frac{\partial \psi}{\partial y} = 1$
bottom : $\psi = 0$ and $\frac{\partial \psi}{\partial y} = 0$
left : $\psi = 0$ and $\frac{\partial \psi}{\partial x} = 0$
right: $\psi = 0$ and $\frac{\partial \psi}{\partial x} = 0$